# PART 1. DATA PREPARATION

In [33]:
import pandas as pd
import requests

Fear & Greed index

In [3]:
# Fear & Greed index

# 1) Download raw Fear & Greed Index data from API

# limit=0 in order to return the full available history
url = "https://api.alternative.me/fng/?limit=0"

resp_fng = requests.get(url)
resp_fng.raise_for_status()

In [4]:
# 2) Parse JSON response and convert to DataFrame
raw = resp_fng.json()
print(raw.keys())

dict_keys(['name', 'data', 'metadata'])


In [5]:
# We only need the 'data' list
data_fng = raw['data']
df_fng = pd.DataFrame(data_fng)
df_fng.head()

,value,value_classification,timestamp,time_until_update
0,17,Extreme Fear,1766016000,31633
1,16,Extreme Fear,1765929600,NaN
2,11,Extreme Fear,1765843200,NaN
3,16,Extreme Fear,1765756800,NaN
4,21,Extreme Fear,1765670400,NaN


In [6]:
# 3) Type conversion and variable cleaning

# Convert UNIX timestamp to pandas datetime
df_fng["date"]      = pd.to_datetime(df_fng["timestamp"], unit="s")

# Convert the index value from string to numeric
df_fng["fng_value"] = pd.to_numeric(df_fng["value"], errors="coerce")

df_fng["fng_class"] = df_fng["value_classification"]
df_fng = df_fng[["date", "fng_value", "fng_class"]]
df_fng.head()

/tmp/ipython-input-3637307011.py:4: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_fng["date"]      = pd.to_datetime(df_fng["timestamp"], unit="s")


,date,fng_value,fng_class
0,2025-12-18,17,Extreme Fear
1,2025-12-17,16,Extreme Fear
2,2025-12-16,11,Extreme Fear
3,2025-12-15,16,Extreme Fear
4,2025-12-14,21,Extreme Fear


In [7]:
df_fng_2017 = df_fng[df_fng["date"] >= "2017-01-01"]

In [8]:
df_fng_2017.to_csv("fear_and_greed_2017.csv", index=False)

Bitcoin Price Data (OHLCV)

In [9]:
import kagglehub
import os

# Download latest version
Bitcoin_dataset_path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")

print("Path to dataset files:", Bitcoin_dataset_path)


#Take the first (and, by set up of the Kaggle page, the only) file in the relevant directory, and read ut to create df_btc_raw

df_btc_raw=pd.read_csv(Bitcoin_dataset_path+"/"+os.listdir(Bitcoin_dataset_path)[0])

print(df_btc_raw.columns)
df_btc_raw.head()

100%|██████████| 98.7M/98.7M [00:00<00:00, 225MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mczielinski/bitcoin-historical-data/versions/451
Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [10]:
# First, make sure it is numeric, then convert to pandas datetime.
df_btc_raw["Timestamp"] = pd.to_numeric(df_btc_raw["Timestamp"], errors="coerce")
df_btc_raw["datetime"] = pd.to_datetime(df_btc_raw["Timestamp"], unit="s")

# Sort by time
df_btc_raw = df_btc_raw.sort_values("datetime")

In [11]:
df_btc_raw.head()

,Timestamp,Open,High,Low,Close,Volume,datetime
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:01:00
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:02:00
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:03:00
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:04:00
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:05:00


In [12]:
# 3) Set datetime as index and resample to daily OHLCV

df_btc_raw = df_btc_raw.set_index("datetime")

# We aggregate the 1-minute OHLCV data into daily OHLCV bars
df_btc_daily = df_btc_raw.resample("D").agg({
    "Open" : "first", # first price of the day
    "High" : "max",   # maximum price of the day
    "Low" : "min",    # minimum price of the day
    "Close" : "last", # last price of the day
    "Volume" : "sum"  # total traded volume during the day
})

In [13]:
# Check missing values for all columns after resampling
print(df_btc_daily.isna().sum())

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [14]:
df_btc_daily = df_btc_daily.reset_index()
df_btc_daily.rename(columns={"datetime": "date"}, inplace=True)
df_btc_daily.head()

,date,Open,High,Low,Close,Volume
0,2012-01-01,4.58,4.84,4.58,4.84,10.000000
1,2012-01-02,4.84,5.00,4.84,5.00,10.100000
2,2012-01-03,5.00,5.32,5.00,5.29,107.085281
3,2012-01-04,5.29,5.57,4.93,5.57,107.233260
4,2012-01-05,5.57,6.46,5.57,6.42,70.328742


In [15]:
# Keep only the columns that are actually needed
df_btc_daily = df_btc_daily[["date", "Open", "High", "Low", "Close", "Volume"]]

# 4) Rename columns to cleaner variable names
df_btc_daily = df_btc_daily.rename(
    columns={
        "Open": "btc_open",
        "High": "btc_high",
        "Low": "btc_low",
        "Close": "btc_close",
        "Volume": "btc_volume",
    }
)

df_btc_daily.head()

,date,btc_open,btc_high,btc_low,btc_close,btc_volume
0,2012-01-01,4.58,4.84,4.58,4.84,10.000000
1,2012-01-02,4.84,5.00,4.84,5.00,10.100000
2,2012-01-03,5.00,5.32,5.00,5.29,107.085281
3,2012-01-04,5.29,5.57,4.93,5.57,107.233260
4,2012-01-05,5.57,6.46,5.57,6.42,70.328742


In [26]:
# 5) Create the dataset truncated to start in 2017:
# Note: Prior to this, Bitcoin market regulations and trading activity were significantly different to the present ones;
# This is why including older data will be uninformative and misleading.

btc_2017 = df_btc_daily[df_btc_daily["date"] >= "2017-01-01"].copy()

# 6) Save the processed daily OHLCV data to CSV files
btc_2017.to_csv("btc_price_2017.csv", index=False)

Google trends (general interest)

In [17]:
# 1) Load raw Google Trends CSV

gt_raw = pd.read_csv("multiTimeline.csv", skiprows = 1)
gt_raw.head()

,월,bitcoin: (전 세계)
0,2007-01,0
1,2007-02,0
2,2007-03,0
3,2007-04,0
4,2007-05,0


In [18]:
# 2) Clean column names and convert date

month_col = gt_raw.columns[0]
interest_col = gt_raw.columns[1]

gt = gt_raw.rename(columns={
    month_col : "date",
    interest_col : "bitcoin_interest"
})

# Convert the date column to pandas datetime
gt['date'] = pd.to_datetime(gt['date'])

# 3) Convert weekly to daily frequency

# forward_fill weekly value for each day
gt_daily = gt.set_index('date').resample("D").ffill()

# reset the index
gt_daily = gt_daily.reset_index()
gt_daily.head()

,date,bitcoin_interest
0,2007-01-01,0
1,2007-01-02,0
2,2007-01-03,0
3,2007-01-04,0
4,2007-01-05,0


In [19]:
gt_2017 = gt_daily[ gt_daily['date'] >= '2017-01-01' ].copy()
gt_2017.to_csv("google_trends_bitcoin_interest_2017.csv", index=False)

On-Chain Metrics

In [27]:
# 1) Helper function

def get_blockchain_chart(chart_type):

# We will use blockchain.com API
  url = f"https://api.blockchain.info/charts/{chart_type}"
  params = {
      "timespan" : "all",
      "format" : "json",
      "sampled" : "false"
  }

  resp = requests.get(url, params = params)
  resp.raise_for_status()

  # Parse JSON response
  raw = resp.json()
  df = pd.DataFrame(raw["values"])

  # Convert UNIX timestamp (seconds) to datetime.date
  df["date"] = pd.to_datetime(df['x'], unit="s").dt.date

  # Rename the value column from 'y' to the chart_type name
  df.rename(columns={"y": chart_type}, inplace=True)

  return df[["date", chart_type]]

# 2) Build on-chain dataset

def get_onchain_blockchain():

  # We will download and merge three available on-chain metrics :
  # - active_addresses : number of unique addresses (n-unique-addresses)
  # - tx_count         : number of transactions (n-transactions)
  # - tx_volume        : estimated transaction volume in BTC (estimated-transaction-volume)

  # Download each series
  df_active = get_blockchain_chart("n-unique-addresses")
  df_tx_count = get_blockchain_chart("n-transactions")
  df_tx_volume  = get_blockchain_chart("estimated-transaction-volume")

  # Merge them on the 'date' column
  # We do how = "outer" to keep all the dates
  df = df_active.merge(df_tx_count, on="date", how="outer")
  df = df.merge(df_tx_volume, on="date", how="outer")

  # Rename columns
  df.rename(columns={
      "n-unique-addresses" : "active_addresses",
      "n-transactions" : "tx_count",
      "estimated-transaction-volume" : "tx_volume"
  }, inplace = True)

  # Sort by time
  df.sort_values("date", inplace=True)
  return df

# Final
df_onchain = get_onchain_blockchain()
df_onchain["date"] = pd.to_datetime(df_onchain["date"])


df_onchain_2017 = df_onchain[df_onchain["date"] >= "2017-01-01"].copy()

df_onchain_2017.to_csv("onchain_2017.csv", index=False)


Major stock indices (S&P 500, Nasdaq, Dow Jones)


In [28]:
import yfinance as yf

In [29]:
# We will get data from Yahoo Finance
# - S&P 500 (^GSPC)
# - Nasdaq Composite (^IXIC)
# - Dow Jones (^DJI)

# We want to get daily data
eq = yf.download(["^GSPC", "^IXIC", "^DJI"], start = "2017-01-01",
                      interval="1d")

# We only need the adjusted close prices
# auto_adjust=True by default
eq_adj = eq["Close"].reset_index().rename(columns={"Date" : "date"})
eq_adj["date"] = pd.to_datetime(eq_adj["date"])

# Rename column names
eq_adj = eq_adj.rename(columns={
    "^GSPC" : "sp_adj",
    "^IXIC" : "nasdaq_adj",
    "^DJI" : "dow_adj"
})

# Compute daily returns for each index
eq_adj["sp_ret"] = eq_adj["sp_adj"].pct_change()
eq_adj["nasdaq_ret"] = eq_adj["nasdaq_adj"].pct_change()
eq_adj["dow_ret"] = eq_adj["dow_adj"].pct_change()

# Final
eq_2017 = eq_adj[eq_adj["date"] >= "2017-01-01"].copy()

eq_2017.to_csv("equity_indices_2017.csv", index=False)

/tmp/ipython-input-3192259696.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eq = yf.download(["^GSPC", "^IXIC", "^DJI"], start = "2017-01-01",
[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-3192259696.py:23: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  eq_adj["sp_ret"] = eq_adj["sp_adj"].pct_change()


Macroeconomic Indicators:

- U.S. Dollar Index (DTWEXBGS)
- 10-Year Treasury Yield (DGS10)
- Consumer Price Index (CPIAUCSL)

In [30]:
# Downloaded datasets manually from fred.stlouisfed.org/series/

# Dollar Index (DTWEXBGS)


dollar_raw_2017 = pd.read_csv("DTWEXBGS_2017.csv")


dollar_raw_2017["date"]=pd.to_datetime(dollar_raw_2017["observation_date"])
dollar_raw_2017["dollar_value"]=pd.to_numeric(dollar_raw_2017["DTWEXBGS"], errors="coerce")

dollar_2017=dollar_raw_2017[["date", "dollar_value"]].copy()

dollar_2017.to_csv("dollar_2017.csv", index=False)

# 10Y Treasury Yield (DGS10)

dgs_raw_2017 = pd.read_csv("DGS10_2017.csv")

dgs_raw_2017["date"]=pd.to_datetime(dgs_raw_2017["observation_date"])
dgs_raw_2017["10Y_yield"]=pd.to_numeric(dgs_raw_2017["DGS10"], errors="coerce")

dgs_2017=dgs_raw_2017[["date", "10Y_yield"]].copy()

dgs_2017.to_csv("dgs_2017.csv", index=False)


# Consumer Price Index (CPIAUCSL)

cpi_raw_2017 = pd.read_csv("CPIAUCSL_2017.csv")

cpi_raw_2017["date"]=pd.to_datetime(cpi_raw_2017["observation_date"])
cpi_raw_2017["cpi"]=pd.to_numeric(cpi_raw_2017["CPIAUCSL"], errors="coerce")

cpi_2017=cpi_raw_2017[["date", "cpi"]].copy()



# But since Consumer Price Index (CPIAUCSL) is monthly data,
# we need to convert it to daily (forward_fill within month)

cpi_daily_2017 = cpi_2017.set_index("date").resample("D").ffill().reset_index()

cpi_daily_2017.to_csv("cpi_daily_2017.csv", index=False)


Now we'll merge the datasets

In [31]:
# Merge
df_2017 = btc_2017.merge(df_fng_2017, on="date", how="outer")
df_2017 = df_2017.merge(df_onchain_2017, on="date", how="outer")
df_2017 = df_2017.merge(gt_2017, on="date", how="outer")
df_2017 = df_2017.merge(dollar_2017, on="date", how="outer")
df_2017 = df_2017.merge(dgs_2017, on="date", how="outer")
df_2017 = df_2017.merge(cpi_daily_2017, on="date", how="outer")
df_2017 = df_2017.merge(eq_2017, on="date", how="outer")


Now we'll handle missing values

In [32]:
# Missing values


# Sort by date
df_2017 = df_2017.sort_values("date").reset_index(drop=True)


# Fear and greed index, fng_value, does not have any values prior to 01/02/2018.
# Since we use it in our analysis, we therefore chose to make 01/02/2018 the first date in the data frame.
df_2018=df_2017[df_2017['date'] >= "01/02/2018"].copy()

cols_to_fill = ['fng_value', 'active_addresses', 'tx_count', 'tx_volume','bitcoin_interest']

# Fill NaNs with rolling mean (window=30)
for col in cols_to_fill:
    df_2018[col] = df_2018[col].fillna(df_2018[col].rolling(window=30, min_periods=1).mean())


# 1) Drop days without a valid Bitcoin close price
df_2018 = df_2018.dropna(subset=["btc_close"])



# 2) Forward_fill other variables  except date
cols_to_ffill = df_2018.columns.drop(["date"])
df_2018[cols_to_ffill] = df_2018[cols_to_ffill].ffill()

# 3) If there are still NaN left, we do back_fill
df_2018[cols_to_ffill] = df_2018[cols_to_ffill].bfill()

print(df_2018.isna().sum())
df_2018.to_csv("bitcoin_2018.csv", index=False)

date                0
btc_open            0
btc_high            0
btc_low             0
btc_close           0
btc_volume          0
fng_value           0
fng_class           0
active_addresses    0
tx_count            0
tx_volume           0
bitcoin_interest    0
dollar_value        0
10Y_yield           0
cpi                 0
dow_adj             0
sp_adj              0
nasdaq_adj          0
sp_ret              0
nasdaq_ret          0
dow_ret             0
dtype: int64
